<details><summary style="display:list-item; font-size:16px; color:blue;">Jupyter Help</summary>
    
Having trouble testing your work? Double-check that you have followed the steps below to write, run, save, and test your code!
    
[Click here for a walkthrough GIF of the steps below](https://static-assets.codecademy.com/Courses/ds-python/jupyter-help.gif)

Run all initial cells to import libraries and datasets. Then follow these steps for each question:
    
1. Add your solution to the cell with `## YOUR SOLUTION HERE ## `.
2. Run the cell by selecting the `Run` button or the `Shift`+`Enter` keys.
3. Save your work by selecting the `Save` button, the `command`+`s` keys (Mac), or `control`+`s` keys (Windows).
4. Select the `Test Work` button at the bottom left to test your work.

![Screenshot of the buttons at the top of a Jupyter Notebook. The Run and Save buttons are highlighted](https://static-assets.codecademy.com/Paths/ds-python/jupyter-buttons.png)

**Setup**

Run the cell below to import the required libraries and initialize our random seed.

Most of this code should be familiar, but observe the last import: `peft` is the Hugging Face library for Parameter Efficient Finetuning. It's got everything we need to conduct LoRA in this exercise.

In [ ]:

import torch
from transformers import AutoTokenizer, TrainingArguments, AutoModelForSequenceClassification, Trainer
from datasets import Dataset
import random
import pandas as pd
from peft import LoraConfig, TaskType, get_peft_model

def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)

set_seed() 

In the cell below, we run all the same setup code we did in the previous finetune. Review it and make sure you remember what's going on, then execute it and proceed to the next checkpoint.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv('imdb_data.csv')
training_set = Dataset.from_pandas(data[data['dataset'] == 'train'])
test_set = Dataset.from_pandas(data[data['dataset'] == 'test'])

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") # this tokenizer will work for our smaller model

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="longest", truncation=True)

tokenized_training_set = training_set.map(tokenize_function, batched=True)
tokenized_test_set = test_set.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./temp_results", # do not change this
    num_train_epochs=3,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    learning_rate=1e-4,
    logging_steps=10,
    save_strategy="no", # do not change this
)

#### Checkpoint 1/3

Now for the unique portion of this training.

We'll first important `LoraConfig`, the `TaskType` class, and the `get_peft_model` function from Hugging Face's remarkable `peft` library.

Then we'll assign the variable `peft_config` below the `LoraConfig` object that we'll use to configure our LoRA model. Pass `LoraConfig` the following arguments:
 - `task_type`: The type of task to configure the model for (e.g., sequence classification, token classification, etc.)
    - for this task, which is sequence classification, use `TaskType.SEQ_CLS` 
 - `inference_mode`: Whether the model is being used for training or inference
    - for this task, which is training, use `False`
 - `r`: The rank of the low-rank matrices that parameterize the LoRA layers
    - Remember that the rank should be a smaller number than any dimension of the weight matrix. A decent start is 16, so enter that.
 - `lora_alpha`: The alpha hyperparameter is a way of indicating how much importance the model should place on the LoRA weights vs. the original weights of the model. Alpha should typically be about double the size of the the rank. So our value should be 32.
 - `lora_dropout`: The dropout rate is used to prevent overfitting. Set it to `0.1`


Don't forget to run the cell and save the notebook before selecting `Test Work`! Open the `Jupyter Help` toggle at the top of the notebook for more details.

In [ ]:
## YOUR SOLUTION HERE ##
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,  # false in training, true when inferring
    r=16,  # Rank of low-rank matrices
    lora_alpha=32,  # analogous to the learning rate in normal GD
    lora_dropout=0.1  # Dropout rate, helps prevent overfitting
)

#### Checkpoint 2/3

True to form, Hugging Face takes care of the rest of the grunt work in setting up our LoRA model. After instantiating the base model, there's only one line of code we need to write.

Under our instantiation of `model`, assign to `lora_model` the result of calling `get_peft_model()`, to which we'll pass our `model` and the `peft_config` we defined above, then execute the cell.

Don't forget to run the cell and save the notebook before selecting `Test Work`! Open the `Jupyter Help` toggle at the top of the notebook for more details.

In [ ]:
## YOUR SOLUTION HERE ##

model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2) 
lora_model = get_peft_model(model, peft_config)


Excellent. In the following cell, we'll define a function that will show us the total number of parameters in the base model, the number of parameters we'll train using LoRA, and the percentage of the total weight count the LoRA weights are. Pass our `lora_model`, execute the cell, and check out how much we've lowered the compute required for this finetune!

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
    
print_trainable_parameters(lora_model)

#### Checkpoint 3/3

OK, pop quiz: Remember how to move a model to the GPU?

In the cell below, move `lora_model` `to` the `device` we specified in the first cell.

Now for another pop quiz: Remember how to train a model with the `Trainer` class?

`Trainer` will take the following named arguments:
- `model`
- `args`
- `train_dataset`
- `eval_dataset`

You should be able to fill these in yourself now. Give it a try!

In the last code block in the cell below, train `lora_model` with `trainer.train()` and evaluate it with `trainer.evaluate()`.

Don't forget to run the cell and save the notebook before selecting `Test Work`! Open the `Jupyter Help` toggle at the top of the notebook for more details.

In [ ]:
## YOUR SOLUTION HERE ##
# put the model on the GPU
lora_model.to(device)

trainer = Trainer(
## YOUR SOLUTION HERE ##
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_training_set,
    eval_dataset=tokenized_test_set
)

## YOUR SOLUTION HERE ##
trainer.train()
trainer.evaluate()




Finally, same as the full finetune, we could save this model for future use using the `model.save_pretrained()` call below.

In [ ]:
model.save_pretrained("./lora_finetunedmodel")